In [12]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil #moving files form one to another
import glob
import tensorflow as tf

In [15]:
# counting the number of images in each folder
# 0 - Braintumor 1- Healthey
ROOT_DIR = 'Brain Tumor Data Set\Brain Tumor Data Set'
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

number_of_images.items()

dict_items([('Brain Tumor', 2513), ('Healthy', 2087)])

In [16]:
len(os.listdir(ROOT_DIR))

2

# We will split the data set into
* 70% for training
* 15% for validation
* 15% for validation
    

In [10]:
#Creating a function for the splitting

def split_data(path,split):
    if not os.path.exists("./"+path):
        os.mkdir("./"+path)

    for dir in os.listdir(ROOT_DIR):
        os.makedirs("./"+path+"/"+ dir) #creating multiple folders
        
        for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR,dir)),size=(math.floor(split*number_of_images[dir])-5),replace=False):
            O = os.path.join(ROOT_DIR,dir,img)
            D = os.path.join("./"+path,dir)
            shutil.copy(O,D)
            os.remove(O)
    else:
        print(f"{path}folder already exists")


 

testfolder already exists


In [17]:
split_data("train",0.7)

trainfolder already exists


In [18]:
split_data("test",0.15)

testfolder already exists


In [19]:
split_data("val",0.15)

valfolder already exists


# Model building

In [36]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras

In [22]:
# CNN model

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))

model.add(Conv2D(filters=36, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25)) # Retain 75% of the data

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 220, 220, 36)   │         5,220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 36)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 108, 108, 64)   │        20,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,537,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,638,245 (21.51 MB)

 Trainable params: 5,638,245 (21.51 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
#compiling the model
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

Data preparation

In [38]:
def preProcessingImages1(path):
    '''
    input : Path
    output : Pre Processed Images
    '''
    

    image_data = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    image = image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')

    return image

In [39]:
path = "train"
train_data = preProcessingImages1(path)

Found 3209 images belonging to 2 classes.


In [40]:
def preProcessingImages2(path):
    '''
    input : Path
    output : Pre Processed Images
    '''
    

    image_data = ImageDataGenerator(rescale=1./255,)
    image = image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')

    return image

In [41]:
path = "test"
test_data = preProcessingImages2(path)

Found 679 images belonging to 2 classes.


In [59]:
path = "val"
val_data = preProcessingImages2(path)

Found 679 images belonging to 2 classes.


In [58]:
# Early stopping and model checkpoint

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# early stopping

es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=3, verbose=1, mode='auto')

# model checkpoint

mc = ModelCheckpoint(filepath='./bestmodel.h5 ./bestmodel.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

cd = [es,mc]

# Model Training


In [ ]:
history = model.fit_generator(generator=train_data, steps_per_epoch=8, epochs=30, validation_data=test_data, validation_steps=len(test_data), callbacks=cd) 